## Bring up a GPU server

In [22]:
from chi import server, context, lease
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [2]:
l = lease.get_lease(f"node-model2-project31") 
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>node-model2-pr…

Lease Details:
Name: node-model2-project31
ID: 23854896-d0f4-4232-8a32-2327f6eecd47
Status: ACTIVE
Start Date: 2025-05-12 01:16:00
End Date: 2025-05-13 01:15:00
User ID: 980559067e994013e0b26ff6dacb9f87dee42ab44864475798f7d35eb764bef6
Project ID: d3c6e101843a4ba79e665ebf59b521a2

Node Reservations:
ID: 52c769de-a27e-427a-8bbb-48b32bda431a, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [3]:
s = server.Server(
    f"node-model2-project31", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server node-model2-project31's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-model2-project31
Id,31cda939-5e71-4a50-bd46-61bc35c563ab
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.67 (v4) Type: fixed MAC: 34:80:0d:de:52:98
Network Name,None
Created At,2025-05-12T01:18:31Z
Keypair,vk2584_nyu_edu-jupyter
Reservation Id,None
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


In [4]:
s.associate_floating_ip()

In [20]:
s.refresh()
s.show(type="widget")

Attribute,node-model2-project31
Id,31cda939-5e71-4a50-bd46-61bc35c563ab
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.67 (v4) Type: fixed MAC: 34:80:0d:de:52:98 IP: 129.114.108.60 (v4) Type: floating MAC: 34:80:0d:de:52:98
Network Name,sharednet1
Created At,2025-05-12T01:18:31Z
Keypair,vk2584_nyu_edu-jupyter
Reservation Id,52c769de-a27e-427a-8bbb-48b32bda431a
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


In [23]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [6]:
s.refresh()
s.show(type="widget")

Attribute,node-model2-project31
Id,31cda939-5e71-4a50-bd46-61bc35c563ab
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.67 (v4) Type: fixed MAC: 34:80:0d:de:52:98 IP: 129.114.108.60 (v4) Type: floating MAC: 34:80:0d:de:52:98
Network Name,sharednet1
Created At,2025-05-12T01:18:31Z
Keypair,vk2584_nyu_edu-jupyter
Reservation Id,52c769de-a27e-427a-8bbb-48b32bda431a
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


## Retrive Code & Set up applications

In [24]:
# clone from the group github
s.execute("git clone https://github.com/ho1447/ML-SysOps_Project.git")

fatal: destination path 'ML-SysOps_Project' already exists and is not an empty directory.


UnexpectedExit: Encountered a bad command exit code!

Command: 'git clone https://github.com/ho1447/ML-SysOps_Project.git'

Exit code: 128

Stdout: already printed

Stderr: already printed



In [8]:
# Set up Docker 
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Verifying Checksum
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:c41088499908a59aae84b0a49c70e86f4731e588a737f1637e73c8c09d995654
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

In [9]:
# Set up AMD GPU
s.execute("sudo apt update; wget https://repo.radeon.com/amdgpu-install/6.3.3/ubuntu/noble/amdgpu-install_6.3.60303-1_all.deb")
s.execute("sudo apt -y install ./amdgpu-install_6.3.60303-1_all.deb; sudo apt update")

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:6 https://download.docker.com/linux/ubuntu noble InRelease
Fetched 256 kB in 9s (28.2 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.


--2025-05-12 01:33:23--  https://repo.radeon.com/amdgpu-install/6.3.3/ubuntu/noble/amdgpu-install_6.3.60303-1_all.deb
Resolving repo.radeon.com (repo.radeon.com)... 23.221.22.214, 23.221.22.215, 2600:1404:6400:25::17de:f148, ...
Connecting to repo.radeon.com (repo.radeon.com)|23.221.22.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16984 (17K) [application/octet-stream]
Saving to: ‘amdgpu-install_6.3.60303-1_all.deb’

     0K .......... ......                                     100% 41.0M=0s

2025-05-12 01:33:24 (41.0 MB/s) - ‘amdgpu-install_6.3.60303-1_all.deb’ saved [16984/16984]





Reading package lists...
Building dependency tree...
Reading state information...
Recommended packages:
  dialog
The following NEW packages will be installed:
  amdgpu-install
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 0 B/17.0 kB of archives.
After this operation, 74.8 kB of additional disk space will be used.
Get:1 /home/cc/amdgpu-install_6.3.60303-1_all.deb amdgpu-install all 6.3.60303-2119913.24.04 [17.0 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Selecting previously unselected package amdgpu-install.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../amdgpu-install_6.3.60303-1_all.deb ...
Unpacking amdgpu-install (6.3.60303-2119913.24.04) ...
Setting up amdgpu-install (6.3.60303-2119913.24.04) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.




Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:3 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble InRelease [5435 B]
Get:4 https://repo.radeon.com/rocm/apt/6.3.3 noble InRelease [2605 B]
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:6 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble/main i386 Packages [12.2 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:8 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble/main amd64 Packages [14.1 kB]
Get:9 https://repo.radeon.com/rocm/apt/6.3.3 noble/main amd64 Packages [60.0 kB]
Hit:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 350 kB in 1s (323 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt li

<Result cmd='sudo apt -y install ./amdgpu-install_6.3.60303-1_all.deb; sudo apt update' exited=0>

In [10]:
s.execute("amdgpu-install -y --usecase=dkms")

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:2 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble InRelease
Hit:3 https://download.docker.com/linux/ubuntu noble InRelease
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:5 https://repo.radeon.com/rocm/apt/6.3.3 noble InRelease
Hit:6 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Fetched 508 kB in 1s (837 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
linux-headers-6.8.0-59-generic is already the newest version (6.8.0-59.61).
linux-headers-6.8.0-59-generic set to manually installed.
The following additional packages will be installed:
  amdgpu-dkms-firmware autoconf automake autotools-dev m4
Suggested packages:
  a

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 27.9 MB in 1s (54.4 MB/s)
Selecting previously unselected package m4.
(Reading database ... 113831 files and directories currently installed.)
Preparing to unpack .../0-m4_1.4.19-4build1_amd64.deb ...
Unpacking m4 (1.4.19-4build1) ...
Selecting previously unselected package autoconf.
Preparing to unpack .../1-autoconf_2.71-3_all.deb ...
Unpacking autoconf (2.71-3) ...
Selecting previously unselected package autotools-dev.
Preparing to unpack .../2-autotools-dev_20220109.1_all.deb ...
Unpacking autotools-dev (20220109.1) ...
Selecting previously unselected package automake.
Preparing to unpack .../3-automake_1%3a1.16.5-1.3ubuntu1_all.deb ...
Unpacking automake (1:1.16.5-1.3ubuntu1) ...
Selecting previously unselected package amdgpu-dkms-firmware.
Preparing to unpack .../4-amdgpu-dkms-firmware_1%3a6.10.5.60303-2119913.24.04_all.deb ...
Unpacking amdgpu-dkms-firmware (1:6.10.5.60303-2119913.24.04) ...
Selecting previously unselected package amdgpu-dkms.
Preparing to unpack .../5-a

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='amdgpu-install -y --usecase=dkms' exited=0>

In [11]:
s.execute("sudo apt -y install rocm-smi")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  librocm-smi64-1
The following NEW packages will be installed:
  librocm-smi64-1 rocm-smi
0 upgraded, 2 newly installed, 0 to remove and 4 not upgraded.
Need to get 362 kB of archives.
After this operation, 1744 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 librocm-smi64-1 amd64 5.7.0-1 [309 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 rocm-smi amd64 5.7.0-1 [52.9 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 362 kB in 0s (811 kB/s)
Selecting previously unselected package librocm-smi64-1.
(Reading database ... 118239 files and directories currently installed.)
Preparing to unpack .../librocm-smi64-1_5.7.0-1_amd64.deb ...
Unpacking librocm-smi64-1 (5.7.0-1) ...
Selecting previously unselected package rocm-smi.
Preparing to unpack .../rocm-smi_5.7.0-1_amd64.deb ...
Unpacking rocm-smi (5.7.0-1) ...
Setting up librocm-smi64-1 (5.7.0-1) ...
Setting up rocm-smi (5.7.0-1) ...
Processing triggers for man-db (2.12.0-4build2) ...
Processing triggers for libc-bin (2.39-0ubuntu8.4) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install rocm-smi' exited=0>

In [12]:
s.execute("sudo usermod -aG video,render $USER")

<Result cmd='sudo usermod -aG video,render $USER' exited=0>

In [15]:
s.execute("sudo reboot")
time.sleep(30)

NoValidConnectionsError: [Errno None] Unable to connect to port 22 on 129.114.108.60

In [16]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.108.60 port 22.


Connection successful


In [17]:
s.execute("rocm-smi")



========================= ROCm System Management Interface =========================
=================================== Concise Info ===================================
GPU  Temp (DieEdge)  AvgPwr  SCLK    MCLK     Fan  Perf  PwrCap  VRAM%  GPU%  
0    29.0c           34.0W   300Mhz  1200Mhz  0%   auto  290.0W    0%   0%    
1    27.0c           34.0W   300Mhz  1200Mhz  0%   auto  290.0W    0%   0%    
=============================== End of ROCm SMI Log ================================


<Result cmd='rocm-smi' exited=0>

In [18]:
# Install nvtop
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

Hit:1 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:3 https://download.docker.com/linux/ubuntu noble InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:5 https://repo.radeon.com/rocm/apt/6.3.3 noble InRelease
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 256 kB in 1s (245 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nvtop
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 180 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 nvtop amd64 3.0.2-1 [62.8 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 62.8 kB in 0s (287 kB/s)
Selecting previously unselected package nvtop.
(Reading database ... 118254 files and directories currently installed.)
Preparing to unpack .../nvtop_3.0.2-1_amd64.deb ...
Unpacking nvtop (3.0.2-1) ...
Setting up nvtop (3.0.2-1) ...
Processing triggers for man-db (2.12.0-4build2) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install nvtop' exited=0>

In [19]:
# Update and install pip
s.execute("""
sudo apt update
sudo apt install -y python3-pip python3-venv
""")

Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://repo.radeon.com/amdgpu/6.3.3/ubuntu noble InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:4 https://repo.radeon.com/rocm/apt/6.3.3 noble InRelease
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 256 kB in 1s (243 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
python3-venv is already the newest version (3.12.3-0ubuntu2).
The following NEW packages will be installed:
  python3-pip python3-wheel
0 upgraded, 2 newly installed, 0 to remove and 4 not upgraded.
Need to get 1370 kB of archives.
After this operation, 7234 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 python3-wheel all 0.42.0-2 [53.1 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 python3-pip all 24.0+dfsg-1ubuntu1.1 [1317 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 1370 kB in 1s (2551 kB/s)
Selecting previously unselected package python3-wheel.
(Reading database ... 118262 files and directories currently installed.)
Preparing to unpack .../python3-wheel_0.42.0-2_all.deb ...
Unpacking python3-wheel (0.42.0-2) ...
Selecting previously unselected package python3-pip.
Preparing to unpack .../python3-pip_24.0+dfsg-1ubuntu1.1_all.deb ...
Unpacking python3-pip (24.0+dfsg-1ubuntu1.1) ...
Setting up python3-wheel (0.42.0-2) ...
Setting up python3-pip (24.0+dfsg-1ubuntu1.1) ...
Processing triggers for man-db (2.12.0-4build2) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='\nsudo apt update\nsudo apt install -y python3-pip python3-venv\n' exited=0>